In [1]:
%pip install --quiet minio scikit-learn joblib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from minio import Minio
from minio.commonconfig    import ENABLED    
from minio.versioningconfig import VersioningConfig 


# NEVER DO THIS IN PRODUCTION, this is just for demo purposes
minio_client =  Minio(
    "minio.kubeflow-user-example-com.svc:9000",
    access_key="minioDev",
    secret_key="minioDevPass123",
    secure=False,
)

bucket_name = "ml-artifacts"
if not minio_client.bucket_exists(bucket_name):
    minio_client.make_bucket(bucket_name)

# idempotently enable versioning
cfg = minio_client.get_bucket_versioning(bucket_name)
if cfg.status != ENABLED:
    minio_client.set_bucket_versioning(bucket, VersioningConfig(ENABLED))
    print("✅  Bucket versioning enabled")


In [3]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import joblib

X, y = load_iris(return_X_y=True)
model = LogisticRegression(max_iter=200)
model.fit(X, y)

artifact_file = "iris_model.pkl"
joblib.dump(model, artifact_file)
print(f"✅ Model trained & saved to {artifact_file}")

✅ Model trained & saved to iris_model.pkl


In [4]:
object_name = artifact_file        # you can add folder prefixes here

minio_client.fput_object(bucket_name, object_name, artifact_file)
print(f"📤 Uploaded {artifact_file}  →  s3://{bucket_name}/{object_name}")

# ─────────────────────────────────────────────
# 5) Verify bucket contents
# ─────────────────────────────────────────────
print("\nObjects now in bucket:")
for obj in minio_client.list_objects(bucket_name):
    print(" •", obj.object_name)

📤 Uploaded iris_model.pkl  →  s3://ml-artifacts/iris_model.pkl

Objects now in bucket:
 • iris_model.pkl
